In [367]:
import dataset as dataset
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.tree import tree

In [368]:
df = pd.read_csv('../data/poker-hand-tiny.csv')
df = df[df.Mano != 9]
df = df[df.Mano != 8]
# df = df[df.Mano != 7]
# df = df[df.Mano != 6]
# df = df[df.Mano != 5]
print(df)

        P1  V1  P2  V2  P3  V3  P4  V4  P5  V5  Mano
0        1   1   1  13   2   4   2   3   1  12     0
1        3  12   3   2   3  11   4   5   2   5     1
2        1   9   4   6   1   4   3   2   3   9     1
3        1   4   3  13   2  13   2   1   3   6     1
4        3  10   2   7   1   2   2  11   4   9     0
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   ...
299995   2  13   4  11   3   4   4   4   2   4     3
299996   3   5   4  12   2   8   4   8   2  13     1
299997   1  10   1   5   1   4   4   9   3  13     0
299998   2   6   3   4   2   3   4   2   2  13     0
299999   4   6   4   1   3   4   1   8   4  11     0

[299994 rows x 11 columns]


In [369]:
# % for train % for test
numTrain = int(0.9*len(df.index))
# splitting dataframe by row index
data_train = df.iloc[:numTrain,:]
data_test = df.iloc[numTrain:,:]
print(data_train)
print(data_test)

        P1  V1  P2  V2  P3  V3  P4  V4  P5  V5  Mano
0        1   1   1  13   2   4   2   3   1  12     0
1        3  12   3   2   3  11   4   5   2   5     1
2        1   9   4   6   1   4   3   2   3   9     1
3        1   4   3  13   2  13   2   1   3   6     1
4        3  10   2   7   1   2   2  11   4   9     0
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   ...
209994   2   7   4   7   3  12   3   5   4   6     1
209995   4  10   4   7   4   8   3   2   1   4     0
209996   4  12   3  13   2   1   4  13   1   1     2
209997   4   6   3  12   2   4   4   1   2   9     0
209998   1   6   3   7   2   3   3  12   2   1     0

[209995 rows x 11 columns]
        P1  V1  P2  V2  P3  V3  P4  V4  P5  V5  Mano
209999   2   1   4   7   1   8   3   4   2   8     1
210000   4   3   1   3   3  11   3   1   2   1     2
210001   2   7   4  12   4  11   2  12   1   8     1
210002   3   1   4   8   3   2   2   8   2   1     2
210003   4   5   4  12   1   3   1   5   3  11     1
...     ..  ..  ..

In [370]:
print(data_train.groupby('Mano').size())
print(data_test.groupby('Mano').size())

Mano
0    105337
1     88707
2     10056
3      4342
4       796
5       407
6       306
7        44
dtype: int64
Mano
0    44952
1    38126
2     4339
3     1896
4      360
5      191
6      117
7       18
dtype: int64


In [371]:
# # SMOTE
# X = data_train.loc[:, data_train.columns != 'Mano']
# y = data_train.Mano
# X_res, y_res = SMOTE().fit_resample(X, y)
# data_train = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)
# print(data_train)



In [372]:
# Ready the Train Data
array_train = data_train.values
data_train = array_train[:, 0:10]
label_train = array_train[:, 10]
print(array_train)
print(data_train)
print(label_train)

# Ready the Test Data
array_test = data_test.values
data_test = array_test[:, 0:10]
label_test = array_test[:, 10]
print(array_test)
print(data_test)
print(label_test)

[[ 1  1  1 ...  1 12  0]
 [ 3 12  3 ...  2  5  1]
 [ 1  9  4 ...  3  9  1]
 ...
 [ 4 12  3 ...  1  1  2]
 [ 4  6  3 ...  2  9  0]
 [ 1  6  3 ...  2  1  0]]
[[ 1  1  1 ...  3  1 12]
 [ 3 12  3 ...  5  2  5]
 [ 1  9  4 ...  2  3  9]
 ...
 [ 4 12  3 ... 13  1  1]
 [ 4  6  3 ...  1  2  9]
 [ 1  6  3 ... 12  2  1]]
[0 1 1 ... 2 0 0]
[[ 2  1  4 ...  2  8  1]
 [ 4  3  1 ...  2  1  2]
 [ 2  7  4 ...  1  8  1]
 ...
 [ 1 10  1 ...  3 13  0]
 [ 2  6  3 ...  2 13  0]
 [ 4  6  4 ...  4 11  0]]
[[ 2  1  4 ...  4  2  8]
 [ 4  3  1 ...  1  2  1]
 [ 2  7  4 ... 12  1  8]
 ...
 [ 1 10  1 ...  9  3 13]
 [ 2  6  3 ...  2  2 13]
 [ 4  6  4 ...  8  4 11]]
[1 2 1 ... 0 0 0]


In [373]:
# print(data_train)
# print(label_train)
# tree.plot_tree(tree.DecisionTreeClassifier().fit(data_train, label_train))

In [374]:
# scaler = StandardScaler()
# # Fit only to the training data
# scaler.fit(data_train)
# # Transform the training and testing data
# data_train = scaler.transform(data_train)
# data_test = scaler.transform(data_test)
# print(data_train)
# print(data_test)

In [375]:
models = [GaussianNB(), tree.DecisionTreeClassifier()]
model_names = [ "Naive Bayes", "Decision Tree"]

In [376]:
# ----------------------------------------------------------------
# Run Each Model
# ----------------------------------------------------------------
for model, name in zip(models, model_names):
    model.fit(data_train, label_train)
    # Predict
    print(data_test)
    prediction = model.predict(data_test)
    # Print Accuracy
    acc = accuracy_score(label_test, prediction)
    print("Accuracy Using", name, ": " + str(acc) + '\n')
    print(classification_report(label_test, prediction))
    print(confusion_matrix(label_test, prediction))


[[ 2  1  4 ...  4  2  8]
 [ 4  3  1 ...  1  2  1]
 [ 2  7  4 ... 12  1  8]
 ...
 [ 1 10  1 ...  9  3 13]
 [ 2  6  3 ...  2  2 13]
 [ 4  6  4 ...  8  4 11]]
Accuracy Using Naive Bayes : 0.4994722163579595

              precision    recall  f1-score   support

           0       0.50      1.00      0.67     44952
           1       0.00      0.00      0.00     38126
           2       0.00      0.00      0.00      4339
           3       0.00      0.00      0.00      1896
           4       0.00      0.00      0.00       360
           5       0.00      0.00      0.00       191
           6       0.00      0.00      0.00       117
           7       0.00      0.00      0.00        18

    accuracy                           0.50     89999
   macro avg       0.06      0.12      0.08     89999
weighted avg       0.25      0.50      0.33     89999

[[44952     0     0     0     0     0     0     0]
 [38126     0     0     0     0     0     0     0]
 [ 4339     0     0     0     0     0     

/home/juan/code/manos-poker/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
